In [1]:
import numpy as np
import matplotlib.pyplot as plt
from   astropy.io import fits as pyfits
import exoplanet as exo

pi = np.pi

SCIT = 58.84876    # Kepler short cadence integration time (sec)
LCIT = 29.4244     # Kepler long cadence integration time (min)

In [2]:
# read in the data
FITSFILE = '/Users/research/Desktop/K00137.01.fits'

with pyfits.open(FITSFILE) as f:
    time  = f['TIME'].data
    flux  = f['FLUX'].data
    error = f['ERROR'].data
    index = f['INDEX'].data
    tts   = f['TTS'].data
    cad   = f['CADENCE'].data
    npts  = f['NPTS'].data
    
    h = f[0].header

# stellar mass and radius
RSTAR  = h['RSTAR']
MSTAR  = h['MSTAR']

# pull planet parameters and make pshape vector
T0   = h['EPOCH']
P    = h['PERIOD']
rp   = h['RPRS']
dur  = h['DURATION']/24   # convert from hours to days
b    = h['IMPACT']
ecc  = h['ECC']
w    = h['W']
u1   = h['U1']
u2   = h['U2']

# note that "esinw" is really "sqrt(e)sin(w)" and likewise for ecosw
esinw = np.sqrt(ecc)*np.sin(w*pi/180)
ecosw = np.sqrt(ecc)*np.cos(w*pi/180)

# this is the parameterization I've been using for my model fits
pshape = np.array([T0, P, rp, 1/dur, b**2, esinw, ecosw, u1, u2])

In [3]:
# convert cadences to strings
cadences = np.zeros_like(cad, dtype='U5')
cadences[cad == 0] = 'short'
cadences[cad == 1] = 'long'

# chop up time, flux, error into stamps
time_stamps  = []
flux_stamps  = []
error_stamps = []

j = 0
for i, n in enumerate(npts):
    time_stamps.append(time[j:j+n])
    flux_stamps.append(flux[j:j+n])
    error_stamps.append(error[j:j+n])
    j += n

In [4]:
# functions for switching parameterizations
def pshape_to_pstarry(pshape, Rstar):
    '''
    Convert pshape vector to a parameterization convenient for starry and exoplanet
        - pshape semimajor axis should be in units of STELLAR radii
        - pstarry semimajor axis should be in units of SOLAR radii
        - rp is really rp/Rstar
        - zeta is inverse transit duration (see Pal 2008 for motivation)
    
    pshape: [T0, P, rp, zeta, b^2, e^1/2*sinw, e^1/2*cosw, u1, u2]
    Rstar: stellar radius [solar radii]    

    -- returns pstarry: [T0, P, rp, a, inc, ecc, w, u1, u2]
    '''
    # pull quantities
    T0, P, rp, zeta, b2, esinw, ecosw, u1, u2 = pshape
    
    # calculate transit duration and impact parameter
    D = 1/zeta
    b = np.sqrt(b2)
    
    # calculate eccentricity and argument
    ecc = esinw**2 + ecosw**2
    w   = np.arctan2(esinw,ecosw)
    
    # calculate ecentricity corrections
    E1 = (1-ecc**2)/(1+ecc*np.sin(w))                                   # Winn 2010, Eq.7 (ecc portion)
    E2 = E1/np.sqrt(1-ecc**2)                                           # Winn 2010, Eq.16
    
    # calculate inclination
    k2 = (1+rp)**2
    acosi = b/E1
    asini = np.sqrt(k2-b2)/np.sin((pi*D)/(P*E2))
    inc   = np.arctan2(asini,acosi)

    # calculate semi-major axis
    a = b/E1/np.cos(inc)*Rstar

    return np.array([T0, P, rp, a, inc, ecc, w, u1, u2])



def pstarry_to_pshape(pstarry, Rstar):
    '''
    Convert pstarry to pshape
        - pshape semimajor axis should be in units of STELLAR radii
        - pstarry semimajor axis should be in units of SOLAR radii
        - rp is really rp/Rstar
        - zeta is inverse transit duration (see Pal 2008 for motivation)

    pstarry: [T0, P, rp, a, inc, ecc, w, u1, u2]
    Rstar: stellar radius [solar radii]    

    -- returns pshape: [T0, P, rp, zeta, b^2, e^1/2*sinw, e^1/2*cosw, u1, u2]

    '''
    # pull quantities
    T0, P, rp, a, inc, ecc, w, u1, u2 = pstarry

    # calculate ecentricity corrections
    E1 = (1-ecc**2)/(1+ecc*np.sin(w))                                   # Winn 2010, Eq.7 (ecc portion)
    E2 = E1/np.sqrt(1-ecc**2)                                           # Winn 2010, Eq.16

    # calculate impact parameter
    b = a*E1*np.cos(inc)                                                # Winn 2010, Eq.7
    b2 = b**2
    
    # calculate transit duration
    k2 = (1+rp)**2                                                      # convenience variable
    D  = (P/pi)*E2*np.arcsin(np.sqrt(k2-b2)/(a*np.sin(inc)))            # Winn 2010, Eq.14

    # calculate eccentricity components
    esinw = np.sqrt(ecc)*np.sin(w)
    ecosw = np.sqrt(ecc)*np.cos(w)

    return np.array([T0, P, rp, 1/D, b2, esinw, ecosw, u1, u2])

In [5]:
# here's our function to calcuate model flux
def calculate_model_flux(pshape, Rstar, tts, time_stamps, cadences, orbittype='Keplerian'):
    '''
    Generate a list of lightcurve models a series of individual transits using starry & exoplanet

    pshape: array of transit parameters -- [T0, P, rp, zeta, b^2, esinw, ecosw, u1, u2]
    Rstar: stellar radius [solar radii]
    tts: list of transit times [BJKD]
    time_stamps: list of time stamps (one per transit)
    cadences: cadence of each transit
    orbittype: 'Keplerian' or 'TTV'

    -- returns modellist: a list of model_stamps
    '''
    # check that vector lengths are consistent
    if len(time_stamps) != len(tts):
        raise ValueError('inconsistent input lengths')
    if len(cadences) != len(tts):
        raise ValueError('inconsistent input lengths')
   
    # convert pshape parameters to a form more easily used by starry
    T0, P, rp, a, inc, ecc, w, u1, u2 = pshape_to_pstarry(pshape, Rstar)
    b = np.sqrt(pshape[4])


    # split up short and long cadence; keep count of how many are in each list
    sc_time = []
    lc_time = []
    sc_pos = [0]
    lc_pos = [0]

    for i, cad in enumerate(cadences):
        if cad == 'short':
            sc_time.append(time_stamps[i])
            sc_pos.append(len(time_stamps[i]))
        elif cad == 'long':
            lc_time.append(time_stamps[i])
            lc_pos.append(len(time_stamps[i]))
        else:
            raise ValueError('all cadences must be "short" or "long"')

    sc_pos = np.cumsum(np.array(sc_pos))
    lc_pos = np.cumsum(np.array(lc_pos))

    # generate the light curve model
    exoSLC = exo.StarryLightCurve([u1, u2])
    if orbittype == 'Keplerian':
        orbit  = exo.orbits.KeplerianOrbit(t0=T0, period=P, a=a, b=b, ecc=ecc, omega=w, r_star=Rstar)
    elif orbittype == 'TTV':
        orbit  = exo.orbits.TTVOrbit(transit_times=[tts], a=a, b=b, ecc=ecc, omega=w, r_star=Rstar)

    sc_model = 1 + exoSLC.get_light_curve(orbit=orbit, r=rp, t=np.hstack(sc_time), texp=SCIT/3600/24, oversample=1).eval()
    lc_model = 1 + exoSLC.get_light_curve(orbit=orbit, r=rp, t=np.hstack(lc_time), texp=LCIT/60/24, oversample=30).eval()

    # workaround because get_light_curve is generating shape (N,1) arrays
    sc_model = np.squeeze(sc_model)
    lc_model = np.squeeze(lc_model)

    # turn it into a list
    j = 0
    k = 0
    N = len(tts)
    modellist = [None]*N
    for i, cad in enumerate(cadences):
        if cad == 'short':
            modellist[i] = sc_model[sc_pos[j]:sc_pos[j+1]]
            j += 1
        elif cad == 'long':
            modellist[i] = lc_model[lc_pos[k]:lc_pos[k+1]]
            k += 1
           
    return modellist


In [ ]:
# if you don't include this line, you'll get an entirely different error
tts = np.asarray(tts, dtype='float')

# now actually calculate it
model_stamps = calculate_model_flux(pshape, RSTAR, tts, time_stamps, cadences, orbittype='TTV')

plt.figure(figsize=(12,6))
plt.plot(np.hstack(time_stamps), np.hstack(flux_stamps), 'k.')
plt.plot(np.hstack(time_stamps), np.hstack(model_stamps), c='orange', lw=2)
plt.show()

/Users/research/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
